# Introduction

Version 2 updated for change in directory locations and permit files. Removed the FracFocus additions.  Just ODNR data.

This notebook contains the code to transform the Excel files available through ODNR (<http://oilandgas.ohiodnr.gov/production>) to a dataset in the same format as Ted's 2016 Production spreadsheet (from a shapefile).
The data are pretty much copied straight over into separate columns for each ODNR file, but aligned by API number.

See the ODNR Production notebook for an alternate assembly of the data.

# Imports

In [1]:
#
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import os
import seaborn as sns

# set datadir to the directory that holds the zipfile
datadir = './sources/'
outdir = './out/'
indir = datadir+'odnr_production_files/'

### ODNR cumulative data files.
utica = datadir+'Utica_052519.xlsx'
marcellus = datadir+'Marcellus_052519.xlsx'

prodcsv = outdir+'production_to_2018Q4.csv'
temp = outdir+'temp.csv'
temp2 = outdir+'temp2.csv'

df_allfn = outdir+'df_all.pkl'

# Fetch the baseline/permit dataset

The baseline file is used for the metadata such as lat/lon, owner, etc.  It comes from two files on the ODNR page:
<http://oilandgas.ohiodnr.gov/shale#SHALE>

In [2]:
####  Read in the ODNR excel sheets

uti = pd.read_excel(utica,header=4,dtype={'API #':'category'})
mar = pd.read_excel(marcellus,header=4,dtype={'API #':'category'})
baseline = pd.concat([uti,mar])
baseline = baseline.dropna()
# rename a few things
baseline.columns = ['PermitDate','County','Township','API','Status','Operator','WellName_Number','SurfLat','SurfLon','EndLat','EndLon']

# print the number of APIs
print(f'Total length = {len(baseline)}')
print(f'Unique API   = {len(baseline["API"].unique())}')
baseline[baseline.API.duplicated()]


Total length = 3162
Unique API   = 3159


,PermitDate,County,Township,API,Status,Operator,WellName_Number,SurfLat,SurfLon,EndLat,EndLon
0,Permit Issued,County,Township,API #,Status,Operator,Well Name & Number,Surface Lat,Surface Long,Endpoint Lat,Endpoint Long
7,04/09/2012,BELMONT,MEAD,34013206530000,Producing,XTO ENERGY INC.,KALDOR 2H,39.9760613,-80.8381833,39.9698310,-80.8196450
8,11/19/2018,BELMONT,RICHLAND,34013206620000,Permitted,XTO ENERGY INC.,COOK UNIT 1H,40.0003351,-80.8905795,39.9882041,-80.8680388


## Filling in the gaps of the metadata

Unfortunately, currently (5/2019) the ODNR permit dataset does not include all wells that are listed in the production dataset.  Therefore, we will pull in metadata directly from the production set (even though it is not as complete) and add it to the baseline set for those wells missing from the baseline.

First, we will pull in all the production data and concat to a single dataframe.  

In [7]:
#  The files from 2011 and 2012 are a different format and order so we need different sets of column names.

colnames1 = ['APINumber','Yr','Q','Operator','County','Township','WellName','WellNum','Oil','Gas','Brine','Days']
colnames2 = ['Yr','Q','Operator','County','APINumber','Township','WellName','Oil','Gas','Brine','Days','FirstDate']

dlst = []


for f in filelist:
    d = pd.read_excel(indir+f)
    if f[:4] in ['2011','2012']:
        # Need to create four quarters out of the yearly data.
        d.columns = colnames2
        #d['period'] = pd.to_period(f)
        d['Year'] = f[:4]
        d['Quarter'] = '1' 
        d['period'] = f[:4]+'Q1'
#         for s in ['Q2','Q3','Q4']:
#             nd = d.copy()
#             nd['period'] = f[:4]+s
#             nd['Quarter'] = s[1]
#             d = pd.concat([d,nd])
#         d['Gas'] = d['Gas']/4   # Spread year's productio across 4 quarters evenly
#         d['Oil'] = d['Oil']/4   # Spread year's productio across 4 quarters evenly
#         d['Brine'] = d['Brine']/4   # Spread year's productio across 4 quarters evenly
    else:
        # The quarterly data doesn't need as much...
        d.columns = colnames1
        d['Year'] = f[:4]
        d['Quarter'] = f[5]
        d['period'] = f[:6].replace('_','Q')
    d = d.filter(['APINumber','Year','Quarter','Operator','County','Township','Wellname','Oil','Gas','Brine','Days','period'])
    
    d.APINumber = d.APINumber.fillna(0).astype('int64').astype(str)
    dlst.append(d)
df_all = pd.concat(dlst,sort=True,ignore_index=True)
df_all.to_pickle(df_allfn)
df_all.head(20)

,APINumber,Brine,County,Days,Gas,Oil,Operator,Quarter,Township,Year,period
0,34067210570100,8937.0,HARRISON,198.0,1523465.0,13472.0,CHESAPEAKE APPALACHIA LLC,1,ARCHER,2011,2011Q1
1,34019220730100,23585.0,CARROLL,206.0,322435.0,12334.0,CHESAPEAKE APPALACHIA LLC,1,AUGUSTA,2011,2011Q1
2,34019220850000,2010.0,CARROLL,5.0,0.0,654.0,CHESAPEAKE APPALACHIA LLC,1,CENTER,2011,2011Q1
3,34019220810100,10263.0,CARROLL,11.0,0.0,818.0,CHESAPEAKE APPALACHIA LLC,1,CENTER,2011,2011Q1
4,34019220740000,2403.0,CARROLL,53.0,137192.0,2167.0,CHESAPEAKE APPALACHIA LLC,1,EAST,2011,2011Q1
5,34099231350100,8389.0,MAHONING,79.0,0.0,758.0,CHESAPEAKE APPALACHIA LLC,1,MILTON,2011,2011Q1
6,34133244200100,1796.0,PORTAGE,20.0,0.0,583.0,CHESAPEAKE APPALACHIA LLC,1,SUFFIELD,2011,2011Q1
7,34019220760100,9102.0,CARROLL,92.0,183142.0,6096.0,CHESAPEAKE APPALACHIA LLC,1,WASHINGTON,2011,2011Q1
8,34019220750000,9519.0,CARROLL,130.0,395290.0,9444.0,CHESAPEAKE APPALACHIA LLC,1,WASHINGTON,2011,2011Q1
9,34019221100000,7217.0,CARROLL,52.0,56364.0,11760.0,CHESAPEAKE EXPLORATION LLC,1,CENTER,2012,2012Q1


In [8]:
df_all = pd.read_pickle(df_allfn)

#print(df_all.head())
API = pd.DataFrame(df_all['APINumber'].unique(),columns=['API'])
mg = pd.merge(baseline,API,left_on='API',right_on='API',how='outer',indicator=True)
#print(mg.head())
# the following df has the API of the not-baseline-included wells.
ext_API = pd.DataFrame(mg[mg['_merge']=='right_only']).filter(['API'],axis=1)
#now let's get the metadata from df_all
df_tmp = df_all.groupby('APINumber',as_index=False)['Operator','County','Township'].last()
meta = pd.merge(df_tmp,ext_API,left_on='APINumber',right_on='API',how='inner')
meta = meta.filter(['API','Operator','County','Township']) #.dropna()
meta

,API,Operator,County,Township
0,0,NaN,NaN,NaN
1,34013209630000,ASCENT RESOURCES UTICA LLC,BELMONT,WHEELING
2,34013211170000,RICE DRILLING D LLC,BELMONT,MEAD
3,34013212350000,GULFPORT APPALACHIA LLC,BELMONT,MEAD
4,34019225470000,CHESAPEAKE EXPLORATION LLC,CARROLL,PERRY
5,34059242760100,PIN OAK ENERGY PARTNERS LLC,GUERNSEY,SPENCER
6,34059243100100,PIN OAK ENERGY PARTNERS LLC,GUERNSEY,SPENCER
7,34059245060000,ECLIPSE RESOURCES I LP,GUERNSEY,MILLWOOD
8,34067211610000,CHEVRON APPALACHIA LLC,HARRISON,MONROE
9,34067212010000,CHESAPEAKE EXPLORATION LLC,HARRISON,ARCHER


In [9]:
#Fill in with NaN 
bl = pd.concat([baseline,meta],sort=False,ignore_index=True)
bl.fillna(np.NaN,inplace=True)
bl.tail(10)

,PermitDate,County,Township,API,Status,Operator,WellName_Number,SurfLat,SurfLon,EndLat,EndLon
3172,NaN,HARRISON,WASHINGTON,34067214140000,NaN,CHEVRON APPALACHIA LLC,NaN,NaN,NaN,NaN,NaN
3173,NaN,JEFFERSON,ISLAND CREEK,34081205310000,NaN,CHESAPEAKE EXPLORATION LLC,NaN,NaN,NaN,NaN,NaN
3174,NaN,MONROE,ADAMS,34111245150000,NaN,Eclipse Resources I LP,NaN,NaN,NaN,NaN,NaN
3175,NaN,MONROE,SWITZERLAND,34111245700000,NaN,ECLIPSE RESOURCES I LP,NaN,NaN,NaN,NaN,NaN
3176,NaN,MONROE,BENTON,34111248230000,NaN,TRIAD HUNTER LLC,NaN,NaN,NaN,NaN,NaN
3177,NaN,NOBLE,MARION,34121244780000,NaN,ANTERO RESOURCES CORPORATION,NaN,NaN,NaN,NaN,NaN
3178,NaN,NOBLE,MARION,34121245320000,NaN,ANTERO RESOURCES CORPORATION,NaN,NaN,NaN,NaN,NaN
3179,NaN,PORTAGE,WINDHAM,34133244400000,NaN,MOUNTAINEER KEYSTONE LLC,NaN,NaN,NaN,NaN,NaN
3180,NaN,WASHINGTON,AURELIUS,34167297200000,NaN,TRIAD HUNTER LLC,NaN,NaN,NaN,NaN,NaN
3181,NaN,WASHINGTON,WATERFORD,34167297610000,NaN,PDC ENERGY INC,NaN,NaN,NaN,NaN,NaN


# Fetch the data; concat

Because the data in the files from ODNR is not exactly consistent, We must name the files by hand and use the name of those files as info for the resulting dataframe.  Specifically, name files downloaded : 'YEAR_Q.extension'.  Keep the same extension as the downloaded file (xls or xslx). Add the name to the "fns" variable below.  This program will use the Year and Q info to label the records in the dataframe.  Note that files for years 2011 and 2012 contain annual data only.

In [10]:
blcop = bl.copy()
fns = ['2011.xls','2012.xls',
      '2013_1.xlsx','2013_2.xlsx','2013_3.xlsx','2013_4.xlsx',
      '2014_1.xls','2014_2.xls','2014_3.xls','2014_4.xls',
      '2015_1.xls','2015_2.xls','2015_3.xls','2015_4.xls',
      '2016_1.xls','2016_2.xls','2016_3.xls','2016_4.xls',
      '2017_1.xls','2017_2.xls','2017_3.xls','2017_4.xls',
      '2018_1.xls','2018_2.xls','2018_3.xls','2018_4.xls' ]

#  The files from 2011 and 2012 are a different format and order so we need different sets of column names.

colnames1 = ['APINumber','Yr','Q','Owner','County','Twp','WellName','WellNum','Oil','Gas','Brine','Days']
colnames2 = ['Yr','Q','Owner','County','APINumber','Twp','WellName-WellNum','Oil','Gas','Brine','Days','FirstDate']

def update_col(indx,cols):
    # add the index to the name of each column (except the first (API))
    new = [cols[0]]
    for c in cols[1:]: # skip the first value, it is the API number
        new.append(c+str(indx))
    return new


for i,f in enumerate(fns):
    d = pd.read_excel(indir+f) #,dtype={'Permit Number':'category'})

    if f[:4] in ['2011','2012']:  # 2011 and 2012 are odd and have to be treated separately
        d.columns = colnames2
        d['Year'] = f[:4]
        kcols = ['APINumber','Year','Oil','Gas','Brine','Days']
        d = d.filter(kcols)
        if f[:4]=='2012':  #need to add '1'
            kcols = update_col(i,kcols)
            d.columns = kcols
        d.APINumber = d.APINumber.fillna(0).astype('int64').astype(str)

    else:  # all the other files
        d.columns = colnames1
        d['Year'] = f[:4]
        d['Quarter'] = f[5]
        kcols = ['APINumber','Year','Quarter','Oil','Gas','Brine','Days']
        d.APINumber = d.APINumber.fillna(0).astype('int64').astype(str)
        # some quarters have more than one record for an API - looks like the owner channges hands
        # mid-quarter.  Summing the duplicates (incl # days) solves the issue.
        d = d.groupby(['APINumber','Year','Quarter'],as_index=False)['Oil','Gas','Brine','Days'].sum()
        d = d.filter(kcols)
        kcols = update_col(i,kcols)
        d.columns = kcols

    #print(f'{f[:6]}: total: {len(d)}, unique: {len(d.APINumber.unique())}' )
    #print(f'{f}\n {d.APINumber.describe()}')
    blcop = blcop.merge(d,how='outer',left_on='API',right_on='APINumber')
    blcop = blcop.drop('APINumber',axis=1)



# Summary columns


In [11]:
summ = df_all.groupby('APINumber',as_index=False)['Oil','Gas','Brine','Days'].sum()
summ.columns = ['APINumber','TotalOil','TotalGas','TotalBrine','TotalDays']
blcop = blcop.merge(summ,how='inner',left_on='API',right_on='APINumber')
blcop = blcop.drop('APINumber',axis=1)

blcop['OilPerDay'] = blcop.TotalOil/blcop.TotalDays
blcop['GasPerDay'] = blcop.TotalGas/blcop.TotalDays
blcop['BrinePerDay'] = blcop.TotalBrine/blcop.TotalDays

#print(blcop[blcop.API.duplicated()])
blcop = blcop.drop_duplicates()

# gpk = pd.read_csv(PeakStats)
# blcop = blcop.merge(gpk,left_index=True,right_index=True,how='inner')
# blcop = blcop.drop('Unnamed: 0',axis=1)
blcop.to_csv(prodcsv)
blcop.head(20)

,PermitDate,County,Township,API,Status,Operator,WellName_Number,SurfLat,SurfLon,EndLat,...,Gas25,Brine25,Days25,TotalOil,TotalGas,TotalBrine,TotalDays,OilPerDay,GasPerDay,BrinePerDay
0,11/02/2011,ASHLAND,CLEAR CREEK,34005241600100,Drilled,DEVON ENERGY PRODUCTION CO LP,EICHELBERGER DAVID 1H,40.9466508,-82.4034777,40.9594639,...,NaN,NaN,NaN,0.00,0.0,23598.00,102.0,0.000000,0.000000,231.352941
1,04/09/2012,BELMONT,MEAD,34013206530000,Producing,XTO ENERGY INC.,KALDOR 2H,39.9760613,-80.8381833,39.9698310,...,30364.0,935.0,79.0,27747.33,477770.0,28292.00,810.0,34.255963,589.839506,34.928395
3,04/09/2012,BELMONT,MEAD,34013206540100,Producing,XTO ENERGY INC.,KALDOR 1H,39.9760339,-80.8381875,39.9667858,...,36374.0,408.0,81.0,0.00,4417681.0,72187.00,1615.0,0.000000,2735.406192,44.697833
4,06/15/2012,BELMONT,KIRKWOOD,34013206570100,Producing,GULFPORT ENERGY CORPORATION,SHUGERT 1-1H,40.0431192,-81.1231952,40.0574876,...,42197.0,334.0,91.0,3547.00,3376091.0,54445.00,1959.0,1.810618,1723.374681,27.792241
5,06/21/2012,BELMONT,WARREN,34013206580100,Producing,GULFPORT ENERGY CORPORATION,SHUGERT 1-12H,40.0326937,-81.1457695,40.0538862,...,108925.0,1530.0,92.0,17681.00,5810035.0,110212.00,1912.0,9.247385,3038.721234,57.642259
6,03/21/2012,BELMONT,WHEELING,34013206600100,Producing,ASCENT RESOURCES UTICA LLC,CAPSTONE HOLDINGS 2H-9,40.1498040,-81.0174046,40.1357399,...,63624.0,217.0,91.0,871.00,2352906.0,33317.94,1450.0,0.600690,1622.693793,22.977890
7,03/07/2013,BELMONT,RICHLAND,34013206610000,Producing,ASCENT RESOURCES UTICA LLC,PORTERFIELD GAS UNIT C 1H-17,40.0889823,-80.9331631,40.1041411,...,79937.0,295.0,88.0,0.00,5346302.0,23041.00,1616.0,0.000000,3308.355198,14.258045
8,07/28/2014,BELMONT,UNION,34013206680100,Producing,ASCENT RESOURCES UTICA LLC,BEDWAY N UNN BL 1H,40.0758974,-81.0237230,40.0867606,...,160679.0,1008.0,90.0,0.00,5029611.0,35509.00,1151.0,0.000000,4369.774978,30.850565
9,09/07/2012,BELMONT,SOMERSET,34013206700100,Producing,GULFPORT ENERGY CORPORATION,STUTZMAN 1-14H,39.8905635,-81.1708906,39.8693232,...,98824.0,1783.0,91.0,422.00,6387796.0,82840.00,1926.0,0.219107,3316.612669,43.011423
10,12/11/2012,BELMONT,SOMERSET,34013206720000,Producing,GULFPORT ENERGY CORPORATION,MCCORT 1-28H,39.9196305,-81.2152106,39.9377172,...,113159.0,1121.0,92.0,2131.00,6420698.0,56685.00,1886.0,1.129905,3404.399788,30.055673
